# GEFS operational ensemble mean wget cmd generator

This notebook generates qsub scripts to download operational GEFS ensemble mean
* Source: https://noaa-gefs-pds.s3.amazonaws.com/

In [1]:
import os
import numpy as np
from datetime import datetime, timedelta

In [ ]:
import sys
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/')
from namelist import *

In [4]:
keys = ['geavg',]

key_leads = []

for i in range(3, 168+3, 3):
    key_leads.append('{:03d}'.format(i))

In [14]:
base = datetime(2020, 1, 1)
date_list = [base + timedelta(days=day) for day in range(366)]

## Fill-in-the-gap version

In [15]:
# check if files exist
date_list_fill = []

for dt in date_list:
    
    dt_str = datetime.strftime(dt, '%Y%m%d')
    filename = camp_dir+'wget_GEFSv12/{}_done.txt'.format(dt_str)
    
    if os.path.isfile(filename):
        continue;
    else:
        date_list_fill.append(dt)
        

In [18]:
L_fill = len(date_list_fill)
print('need to download {} files'.format(L_fill))
#date_list_fill = date_list_fill[::-1]

need to download 1 files


## Generate qsub scripts

In [20]:
N = int(L_fill/4) + 1
index = np.arange(0, L_fill, N)

for i_, i_start in enumerate(index):

    f = open(work_dir+'qsub/wget_gefs_{:03d}.sh'.format(i_), 'w') 
    
    heads = '''
    #!/bin/bash -l
    
    #PBS -N wget_gefs
    #PBS -A NAML0001
    #PBS -l walltime=23:59:59
    #PBS -l select=1:ncpus=4:mem=12GB
    #PBS -q casper
    #PBS -o wget_gefs.log
    #PBS -e wget_gefs.err
    
    cd {}wget_GEFSv12/
    
    '''.format(camp_dir)
    
    print(heads, file=f)
    
    for i in range(i_start, i_start+N, 1):
        if i < L_fill:
            dt = date_list_fill[i]
            dt_str = datetime.strftime(dt, '%Y%m%d')
            
            for member in keys:
                for ff in key_leads:
                    download_link = 'https://noaa-gefs-pds.s3.amazonaws.com/gefs.{}/00/atmos/pgrb2sp25/{}.t00z.pgrb2s.0p25.f{}'.format(
                        dt_str, member, ff)
                    
                    save_name = '{}_00_{}.t00z.pgrb2s.0p25.f{}'.format(dt_str, member, ff)
                    
                    print('wget -L -O {} {}'.format(save_name, download_link), file=f)
                    
            print('touch {}_done.txt'.format(dt_str), file=f)
    
    f.close()

f = open(work_dir+'wget_gefs_all.sh', 'w')

for i_, i_start in enumerate(index):
    print('qsub wget_gefs_{:03d}.sh'.format(i_), file=f)
    
f.close()